Inspiration and references --->

# Collaborative filtering on MovieLens using matrix factorization

## Matrix Factorization with bias
$U$ and $V$ are the embedding matrix for users and movies respectively. We are adding $u^0$ which is a bias vector of dimension $n_u$ and $v^0$ which is a bias vector of dimension $n_m$. The predicted rating for $i$ user and $j$ movie is given by the below equation:

$$\hat{y}_{ij} = u_{0i} + v_{0j} + u_i \cdot v_j  $$ 

Vectorized version of gradient descent equation:
   $$1.\: u^0 = u^0 - \eta * \nabla E(u^0) \;where \; \nabla E(u^0)=-2*np.sum((Y-UV^T-u^0-v^{0T})*R,axis = 0)/N$$ <br/>
   $$2.\: v^0 = v^0 - \eta * \nabla E(v^0) \;where \; \nabla E(v^0)=-2*(np.sum((Y-UV^T-u^0-v^{0T})*R,axis = 1))^T/N$$<br/>
   
   $$3. \: U = U - \eta * \nabla E(U)\;where \; \nabla E(U)=-2*((Y-UV^T-u^0-v^{0T})*R)V)/N$$<br/>
   $$4. \:V = V - \eta * \nabla E(V) \;where \; \nabla E(V)=-2*((Y-UV^T-u^0-v^{0T})*R)^TU)/N$$<br/>
   
   where $R$ is the binarized version of $Y$

In [1]:
import numpy as np
import pandas as pd
from pdb import set_trace

### Encoding data 

In [2]:
def proc_col(col):
    """
    Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [3]:
def encode_data(df):
    """
    Encodes rating data with continous user and movie ids
    
    Arguments:
      df: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies
    """
    _,user_col,num_users = proc_col(df.userId)
    _,movie_col,num_movies = proc_col(df.movieId)
    df.userId = user_col
    df.movieId = movie_col
    return df, num_users, num_movies

In [4]:
def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    u2idx,_,_ = proc_col(df_train.userId)
    m2idx,_,_= proc_col(df_train.movieId)
    df_val = df_val.loc[df_val['userId'].isin(u2idx.keys())]
    df_val = df_val.loc[df_val['movieId'].isin(m2idx.keys())]
    df_val.userId = df_val.userId.apply(lambda x: u2idx[x])
    df_val.movieId = df_val.movieId.apply(lambda x: m2idx[x])
    return df_val

### Initializing embeddings matrices

In [5]:
def create_embedings(n, K):
    """ 
    Create a numpy random matrix of shape n, K which will represent embedding matrix
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embedding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

### Data

Movie Lens data available here: http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [6]:
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))

20205 19507


In [7]:
K = 50 #dimension of embedding vector
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
bias_user = create_embedings(num_users,1)
bias_movie = create_embedings(num_movies,1)

### Sparse matrix

In [15]:
from scipy import sparse
def df2matrix(df, nrows, ncols, column_name="rating"):
    """ 
    Returns a sparse matrix constructed from a dataframe. Sparse matrix helps in faster calculation
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [34]:
def sparse_multiply(df, emb_user, emb_movie,bias_user,bias_movie):
    """This function returns U*V^T element wise multi by R as a sparse matrix.
    
    It avoids creating the dense matrix U*V^T
    """
    df["Prediction"] = np.squeeze(np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)[:,None]
                                  +bias_user[df['userId'].values]+bias_movie[df['movieId'].values])
    return df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="Prediction")

IndentationError: unexpected indent (<ipython-input-34-a947273f44ec>, line 7)

### Calculating the cost

In [37]:
def cost(df, emb_user, emb_movie, bias_user, bias_movie):
    """
    Calculate the prediction and then calculate the MSE error
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    N = len(Y.data)
    df["Prediction"] = np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)[:,None]+bias_user[df['userId'].values]+bias_movie[df['movieId'].values]
    error = np.sum(np.square(df.Prediction - df.rating))/N
    return error

### Gradient calculation

In [38]:
def gradient_regularized(df, Y, emb_user, emb_movie,bias_user,bias_movie,lambd):
    """ 
    Computes the gradient. 
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embeddings for users
      emb_movie: embeddings for movies
      bias_user: bias for user
      bias_movie: bias for movie
      
    Returns:
      d_emb_user: gradient wrt user embedding
      d_emb_movie: gradient wrt movie embedding
      d_bias_user: gradient wrt user bias
      d_bias_movie: gradient wrt movie bias
      lambd: regularization parameter
      
    """
    N = len(Y.data) #non-empty count
    Y_predict= sparse_multiply(df, emb_user, emb_movie, bias_user, bias_movie) #prediction sparse matrix
    delta = (Y - Y_predict).toarray()
    
    #calculating the gradient
    d_emb_user = -2*np.dot(delta,emb_movie)/N + 2*lambd*emb_user/N
    d_emb_movie = -2*np.dot(delta.T,emb_user)/N +2*lambd*emb_movie/N
    d_bias_user = -2*np.sum(delta,axis=1)/N
    d_bias_movie = -2*np.sum(delta,axis=0).T/N
    
    return d_emb_user,d_emb_movie,d_bias_user,d_bias_movie

### Gradient descent 

In [39]:
# you can use a for loop to iterate through gradient descent
def gradient_descent_reg(df, emb_user, emb_movie,bias_user,bias_movie,iterations=100, learning_rate=0.01, df_val=None,lambd=.01,mom_fact = 0.9):
    """ 
    Computes gradient descent with momentum (mom_fact =.9 (default)) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns: Learnt parameters
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    bias_user: bias for user
    bias_movie: bias for movie
    """
#     set_trace()
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    # Weighted gradient initialization
    grad_u_moment,grad_m_moment ,grad_ub_moment,grad_mb_moment= gradient_regularized(df,Y,emb_user,emb_movie,bias_user,bias_movie,lambd)

    for i in range(iterations):
        grad_user,grad_movie,grad_user_bias,grad_movie_bias = gradient_regularized(df,Y,emb_user,emb_movie,bias_user,bias_movie,lambd)
        
        #Weighted gradient calculation
        grad_u_moment = mom_fact*grad_u_moment+(1-mom_fact)*grad_user #user embedding
        grad_m_moment = mom_fact*grad_m_moment + (1-mom_fact)*grad_movie #movie embedding
        grad_ub_moment = mom_fact*grad_ub_moment+(1-mom_fact)*grad_user_bias #user bias
        grad_mb_moment = mom_fact*grad_mb_moment + (1-mom_fact)*grad_movie_bias  #movie bias      
        
        #weight update
        emb_user = np.array(np.subtract(emb_user,learning_rate*grad_u_moment))
        emb_movie = np.array(np.subtract(emb_movie,learning_rate*grad_m_moment))
        bias_user = np.array(np.subtract(bias_user,learning_rate*grad_ub_moment))
        bias_movie = np.array(np.subtract(bias_movie,learning_rate*grad_mb_moment))
        
        if df_val is not None and i%50 ==0:
            print("Training cost:",cost(df, emb_user, emb_movie,bias_user,bias_movie))
            print("Validation cost:",cost(df_val, emb_user, emb_movie,bias_user,bias_movie))

    return emb_user, emb_movie, bias_user,bias_movie

### Training

In [40]:
emb_user, emb_movie, bias_user, bias_movie = gradient_descent_reg(df_train, emb_user, emb_movie, bias_user, bias_movie, iterations=1, learning_rate=1, df_val=df_val)

ValueError: operands could not be broadcast together with shapes (79799,671) (79799,8442) 